In [1]:
import cv2
import numpy as np
import os
%matplotlib inline

In [2]:
side='right'

In [3]:
base_path = '/home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/'
tempFileNames = os.listdir(base_path)
fileNames = []
for fileName in tempFileNames:
    fileNames.append(base_path+str(fileName))

In [32]:
def det(a, b):
    return a[0] * b[1] - a[1] * b[0]

def line_intersection(line1, line2):
    x_diff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    y_diff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    div = det(x_diff, y_diff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, x_diff) / div
    y = det(d, y_diff) / div

    return [x, y]

def find_intersections(lines, height, width):
    # print("This image's size:", height, width)
    intersections = []
    for i, line_1 in enumerate(lines):
        for line_2 in lines[i + 1:]:
            if not line_1 == line_2:
                intersection = line_intersection(line_1, line_2)
                if intersection:
                    if intersection[0]<0 or intersection[0]>width or intersection[1]<0 or intersection[1]>height:
                        intersections.append(intersection)
                        # print("Intersection Point:"+str(intersection[0])+", "+str(intersection[1]))

    return intersections

In [33]:
def draw_vertical_lines(image, side):
    img = image
    selectedLines = []
    selectedLinesParams = []
    linesFound = False
    BlueRedMask = 100
    
    while linesFound == False:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, (35, BlueRedMask, BlueRedMask), (70, 255,255))
        imask = mask>0
        green = np.zeros_like(img, np.uint8)
        green[imask] = img[imask]
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        v = np.median(green)
        sigma = 0.33
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        cv2.imwrite('green.jpg', green)
        edges = cv2.Canny(green,150,250,apertureSize = 3) 
        minLineLength = 1
        maxLineGap = 1250
        lines = cv2.HoughLines(edges,1,np.pi/180, 200)
        if lines.any():
            if len(lines) > 2:  
                linesFound = True  
            else: 
                BlueRedMask -= 10

    for line in lines:
        rho,theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),1)

    linesFound = False
    
    if side == 'left':
        angleMaxLimit = 20
        angleMinLimit = 70
    else:
        angleMaxLimit = 150
        angleMinLimit = 105

    rLimit = 300
    while linesFound == False:
        for line in lines: 
            for r,theta in line:
                isLineValid = True
                a = np.cos(theta) 
                b = np.sin(theta)
                
                if (theta * 180 * 7 / 22) > angleMinLimit and (theta * 180 * 7 / 22) < angleMaxLimit:
                    #print(theta * 180 * 7 / 22 , r)
                    x0 = a*r 
                    y0 = b*r 
                    x1 = int(x0 + 1000*(-b)) 
                    y1 = int(y0 + 1000*(a)) 
                    x2 = int(x0 - 1000*(-b)) 
                    y2 = int(y0 - 1000*(a))
                    
                    #cv2.line(image,(x1,y1), (x2,y2), (0,0,255),1)
                    
                    
                    if len(selectedLines) > 0: 
                        for lineParams in selectedLinesParams:
                            #print(abs(lineParams[0] - r))
                            if abs(lineParams[0] - r) < rLimit:
                                isLineValid = False
                        for selectedLine in selectedLines:
                            if not line_intersection(selectedLine, [[x1,y1],[x2,y2]]):
                                isLineValid = False
                        if [[x1,y1],[x2,y2]] in selectedLines or [[x2,y2],[x1,y1]] in selectedLines:
                            isLineValid = False
                    if isLineValid:
                        #print("P")
                        selectedLines.append([[x1,y1],[x2,y2]])
                        print(x1,y1,x2,y2)
                        selectedLinesParams.append([r, theta])
                        cv2.line(img,(x1,y1), (x2,y2), (255,0,0),1)
                        cv2.putText(img, str((theta * 180 * 7 / 22)) ,(int((x2))  ,  int((y2))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
                        cv2.putText(img, str((theta * 180 * 7 / 22)) ,(int((x1))  ,  int((y1))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
        if len(selectedLines) < 2:
            if rLimit >= 75:
                rLimit -= 10
            else:
                angleMinLimit -= 1
                angleMaxLimit += 1
                rlimit = 100
        else:
            #for par in selectedLinesParams:
            #	print( "Angle", (par[1]*180*7)/22)
            linesFound = True



    return img

In [34]:
def get_intersections(lineSet, height, width):
    intersections = find_intersections(lineSet, height, width)
    vanishingPointX = 0.0
    vanishingPointY = 0.0
    for point in intersections:
        vanishingPointX += point[0]
        vanishingPointY += point[1]
    # print(selectedLines)
    if len(intersections)!=0:
        return (vanishingPointX/len(intersections), vanishingPointY/len(intersections))
    else:
        return (0, 0)

In [35]:
def draw_intersections(lineSet, height, width):
    intersections = find_intersections(lineSet, height, width)
    return intersections

In [36]:
def resizeImage(img, plusVP, minusVP):
    # plusVanishPointInte = (int(plusVP[0]), int(plusVP[1]))
    # minusVanishPointInte = (int(minusVP[0]), int(minusVP[1]))
    pad = 50
    height, width, _ = img.shape
    # print(height, width)
    # if plusVanishPointInte[0]<0 or minusVanishPointInte[0]<0:
    if plusVP[0] < minusVP[0]:
        left = plusVP[0]
        right = minusVP[0]
    else:
        left = minusVP[0]
        right = minusVP[0]
    if plusVP[1] < minusVP[1]:
        top = plusVP[1]
        bottom = minusVP[1]
    else:
        top = minusVP[1]
        bottom = plusVP[1]

    startWidth = 0
    sizeWidth = width
    startHeight = 0
    sizeHeight = height

    if left<0:
        startWidth = -left + pad
    if top<0:
        startHeight = -top + pad
    if right > width:
        sizeWidth = right
    if bottom > height:
        sizeHeight = bottom
    # Data_type = object
    newImage = np.zeros((startHeight+sizeHeight+pad, startWidth+sizeWidth+pad, 3),np.uint8)
    # print(img[0, 3].dtype)
    for local_width in range(width):
        for local_height in range(height):
            # print(img[local_height, local_width].dtype)
            resizedHeight = local_height+startHeight
            resizedWidth = local_width+startWidth
            newImage[resizedHeight][resizedWidth] = img[local_height][local_width]
    
    return newImage, startWidth, startHeight

In [43]:
def draw_vertical_lines_Pro(image):
    img = image
    height, width, _ = img.shape
    selectedLines = []
    selectedLinesParams = []
    linesFound = False
    BlueRedMask = 100
    
    while linesFound == False:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, (35, BlueRedMask, BlueRedMask), (70, 255,255))
        imask = mask>0
        green = np.zeros_like(img, np.uint8)
        green[imask] = img[imask]
        gray = cv2.cvtColor(green, cv2.COLOR_HSV2BGR)
        gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
        blurImage = cv2.GaussianBlur(gray, (5, 5), 1)
        edges = cv2.Canny(blurImage,150,250,apertureSize = 3) 
        minLineLength = 1
        maxLineGap = 1250
        lines = cv2.HoughLinesP(edges,1,np.pi/180, 100, None, 20, 2)
        if lines.any():
            if len(lines) > 2:  
                linesFound = True  
            else: 
                BlueRedMask -= 10
    linePlus=[]
    lineMinus=[]
    for line in lines:
        # print(line)
        # rho,theta = line[0]
        # a = np.cos(theta)
        # b = np.sin(theta)
        # x0 = a*rho
        # y0 = b*rho
        # x1 = int(x0 + 1000*(-b))
        # y1 = int(y0 + 1000*(a))
        # x2 = int(x0 - 1000*(-b))
        # y2 = int(y0 - 1000*(a))
        x1, y1, x2, y2 = line[0]
        dist = (x2-x1, y2-y1)
        if dist[0]*dist[1]>=0:
            linePlus.append([(x1,y1),(x2,y2)])
        else:
            lineMinus.append([(x1,y1),(x2,y2)])
        # x1 -= 10*dist[0]
        # y1 -= 10*dist[1]
        # x2 += 10*dist[0]
        # y2 += 10*dist[1]
        # (x1, y1) = (x1, y1) - 5*dist
        # (x2, y2) = (x2, y2) + 5*dist

        # cv2.line(img,(x1,y1),(x2,y2),(255,255,0),5)
    plusVanishPoint=(0,0)
    minusVanishPoint=(0,0)
    if len(linePlus)!=0:
        plusVanishPoint = get_intersections(linePlus, height, width)
    if len(lineMinus)!=0:
        minusVanishPoint = get_intersections(lineMinus, height, width)
    # print(plusVanishPoint)

    plusVanishPointInte = (int(plusVanishPoint[0]), int(plusVanishPoint[1]))
    minusVanishPointInte = (int(minusVanishPoint[0]), int(minusVanishPoint[1]))

    resizeImg, leftCover, topCover = resizeImage(img, plusVanishPointInte, minusVanishPointInte)
    newplusVPx=plusVanishPointInte[0] + leftCover
    newplusVPy=plusVanishPointInte[1] + topCover
    newminusVPx=minusVanishPointInte[0] + leftCover
    newminusVPy=minusVanishPointInte[1] + topCover
    
    plusIntersectPoints = draw_intersections(linePlus, height, width)
    minusIntersectPoints = draw_intersections(lineMinus, height, width)

    # for point in plusIntersectPoints:
    #     # pointX = int(point[0]+leftCover)
    #     # pointY = int(point[1]+topCover)
    #     newp = (int(point[0]), int(point[1]))
    #     # print("Point is "+str(newp[0])+", "+str(newp[1]))
    #     cv2.line(img, newp, newp, (100, 100, 100), 5)
        
    # for point in minusIntersectPoints:
    #     # pointX = int(point[0]+leftCover)
    #     # pointY = int(point[1]+topCover)
    #     newp = (int(point[0]), int(point[1]))
    #     # print("Point is "+str(newp[0])+", "+str(newp[1]))
    #     cv2.line(img, newp, newp, (200, 200, 0), 5)
    # return img


    for point in plusIntersectPoints:
        pointX = int(point[0]+leftCover)
        pointY = int(point[1]+topCover)
        cv2.line(resizeImg, (pointX, pointY), (pointX, pointY), (100, 100, 100), 5)
        
    for point in minusIntersectPoints:
        pointX = int(point[0]+leftCover)
        pointY = int(point[1]+topCover)
        cv2.line(resizeImg, (pointX, pointY), (pointX, pointY), (200, 200, 0), 5)
        
    
    # if plusVanishPointInte[0]<0:
    #     newplusVPx += leftCover        
    # if minusVanishPointInte[0]<0:
    #     newminusVPx += leftCover        
    # if plusVanishPointInte[1]<0:
    #     newplusVPx += topCover
    # if minusVanishPointInte[1]<0:
    #     newplusVPy += topCover
    newplusVP = (newplusVPx, newplusVPy)
    newminusVP = (newminusVPx, newminusVPy)
    if plusVanishPoint!=(0,0):
        cv2.line(resizeImg, newplusVP, newplusVP, (100, 100, 100), 30)
        cv2.putText(resizeImg, "PlusVP"+str(plusVanishPoint[0])+", "+str(plusVanishPoint[1]),newplusVP, cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    if minusVanishPoint!=(0,0):
        cv2.line(resizeImg, newminusVP, newminusVP, (200, 200, 200), 30)
        cv2.putText(resizeImg, "MinusVP"+str(minusVanishPoint[0])+", "+str(minusVanishPoint[1]),newminusVP, cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    

    if len(linePlus) > len(lineMinus):
        selectedLines = linePlus
        cv2.putText(resizeImg, "Plus is selected",(50, 50) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    else:
        selectedLines = lineMinus
        cv2.putText(resizeImg, "Minus is selected",(50, 50) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    for line in selectedLines:
        dist = (line[1][0]-line[0][0], line[1][1]-line[0][1])
        # newlineX=[]
        # newlineY=[]
        newlineX=(leftCover+line[0][0] - 10*dist[0], topCover+line[0][1] - 10*dist[1])
        # newlineX.append()
        newlineY=(leftCover+line[1][0] + 10*dist[0], topCover+line[1][1] + 10*dist[1])
        # newlineY.append()
        cv2.line(resizeImg, newlineX, newlineY, (255, 255, 0), 5)
        
    return resizeImg


    # if plusVanishPoint!=(0,0):
    #     cv2.line(img, plusVanishPointInte, plusVanishPointInte, (100, 100, 100), 10)
    #     cv2.putText(img, "PlusVP"+str(plusVanishPoint[0])+str(plusVanishPoint[1]),plusVanishPointInte, cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    # if minusVanishPoint!=(0,0):
    #     cv2.line(img, minusVanishPointInte, minusVanishPointInte, (200, 200, 200), 10)
    #     cv2.putText(img, "MinusVP"+str(minusVanishPoint[0])+str(minusVanishPoint[1]),minusVanishPointInte, cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    

    # if len(linePlus) > len(lineMinus):
    #     selectedLines = linePlus
    #     cv2.putText(img, "Plus is selected",(50, 50) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    # else:
    #     selectedLines = lineMinus
    #     cv2.putText(img, "Minus is selected",(50, 50) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    # for line in selectedLines:
    #     dist = (line[1][0]-line[0][0], line[1][1]-line[0][1])
    #     # newlineX=[]
    #     # newlineY=[]
    #     newlineX=(line[0][0] - 10*dist[0], line[0][1] - 10*dist[1])
    #     # newlineX.append()
    #     newlineY=(line[1][0] + 10*dist[0], line[1][1] + 10*dist[1])
    #     # newlineY.append()
    #     cv2.line(img, newlineX, newlineY, (255, 255, 0), 5)
    
    # return img

    # linesFound = False
    
    # if side == 'left':
    #     angleMaxLimit = 20
    #     angleMinLimit = 70
    # else:
    #     angleMaxLimit = 150
    #     angleMinLimit = 105

    # rLimit = 300
    # while linesFound == False:
    #     for line in lines: 
    #         for r,theta in line:
    #             isLineValid = True
    #             a = np.cos(theta) 
    #             b = np.sin(theta)
                
    #             if (theta * 180 * 7 / 22) > angleMinLimit and (theta * 180 * 7 / 22) < angleMaxLimit:
    #                 #print(theta * 180 * 7 / 22 , r)
    #                 x0 = a*r 
    #                 y0 = b*r 
    #                 x1 = int(x0 + 1000*(-b)) 
    #                 y1 = int(y0 + 1000*(a)) 
    #                 x2 = int(x0 - 1000*(-b)) 
    #                 y2 = int(y0 - 1000*(a))
                    
    #                 #cv2.line(image,(x1,y1), (x2,y2), (0,0,255),1)
                    
                    
    #                 if len(selectedLines) > 0: 
    #                     for lineParams in selectedLinesParams:
    #                         #print(abs(lineParams[0] - r))
    #                         if abs(lineParams[0] - r) < rLimit:
    #                             isLineValid = False
    #                     for selectedLine in selectedLines:
    #                         if not line_intersection(selectedLine, [[x1,y1],[x2,y2]]):
    #                             isLineValid = False
    #                     if [[x1,y1],[x2,y2]] in selectedLines or [[x2,y2],[x1,y1]] in selectedLines:
    #                         isLineValid = False
    #                 if isLineValid:
    #                     #print("P")
    #                     selectedLines.append([[x1,y1],[x2,y2]])
    #                     print(x1,y1,x2,y2)
    #                     selectedLinesParams.append([r, theta])
    #                     cv2.line(img,(x1,y1), (x2,y2), (255,0,0),1)
    #                     cv2.putText(img, str((theta * 180 * 7 / 22)) ,(int((x2))  ,  int((y2))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    #                     cv2.putText(img, str((theta * 180 * 7 / 22)) ,(int((x1))  ,  int((y1))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
    #     if len(selectedLines) < 2:
    #         if rLimit >= 75:
    #             rLimit -= 10
    #         else:
    #             angleMinLimit -= 1
    #             angleMaxLimit += 1
    #             rlimit = 100
    #     else:
    #         #for par in selectedLinesParams:
    #         #	print( "Angle", (par[1]*180*7)/22)
    #         linesFound = True


In [44]:
for file_itr in range(len(fileNames)):
    if not tempFileNames[file_itr].endswith("jpg"):
        continue
    print(tempFileNames[file_itr])
    # vertical_vanishing_point = get_vertical_vanishing_point(imageForVanishingPoints, goalDirection)
    image = cv2.imread(fileNames[file_itr])
    # cv2.imshow("Test", image)
    # img1 = draw_vertical_lines(image, side)
    height, width, _ = image.shape
    # print("Image's shape is "+str(height)+", "+str(width))
    img2 = draw_vertical_lines_Pro(image)
    cv2.imwrite("./graytest4/"+tempFileNames[file_itr], img2)
    # if file_itr>=10:
    #     break

340.jpg
470.jpg
125.jpg
52.jpg
321.jpg
274.jpg
120.jpg
308.jpg
406.jpg
313.jpg
64.jpg
365.jpg
208.jpg
304.jpg
172.jpg
115.jpg
460.jpg
486.jpg
370.jpg
202.jpg
195.jpg
369.jpg
61.jpg
215.jpg
190.jpg
184.jpg
134.jpg
218.jpg
442.jpg
7.jpg
296.jpg
34.jpg
339.jpg
88.jpg
366.jpg
214.jpg
479.jpg
292.jpg
122.jpg
439.jpg
149.jpg
422.jpg
249.jpg
50.jpg
39.jpg
436.jpg
298.jpg
185.jpg
36.jpg
226.jpg
242.jpg
6.jpg
235.jpg
257.jpg
262.jpg
467.jpg
280.jpg
8.jpg
91.jpg
92.jpg
124.jpg
288.jpg
66.jpg
209.jpg
417.jpg
15.jpg
16.jpg
331.jpg
104.jpg
355.jpg
438.jpg
360.jpg
289.jpg
419.jpg
239.jpg
432.jpg
253.jpg
231.jpg
282.jpg
10.jpg
310.jpg
169.jpg
266.jpg
350.jpg
311.jpg
456.jpg
74.jpg
394.jpg
389.jpg
377.jpg
40.jpg
435.jpg
65.jpg
335.jpg
222.jpg
484.jpg
167.jpg
437.jpg
152.jpg
224.jpg
322.jpg
387.jpg
472.jpg
56.jpg
182.jpg
452.jpg
58.jpg
352.jpg
12.jpg
210.jpg
431.jpg
87.jpg
70.jpg
217.jpg
411.jpg
317.jpg
294.jpg
48.jpg
63.jpg
1.jpg
82.jpg
57.jpg
161.jpg
381.jpg
303.jpg
329.jpg
140.jpg
269.jpg
101.jpg
32

In [31]:
base_path_img = '/home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/340.jpg'
img = cv2.imread(base_path_img)
height, width, _ = img.shape
print("Image's shape is "+str(height)+", "+str(width))
cv2.line(img, (-1962, 271), (-1962, 271), (200, 200, 0), 30)
cv2.line(img, (-1324, 271), (-1324, 271), (200, 200, 0), 30)
cv2.line(img, (266, 1318), (266, 1318), (200, 200, 0), 30)
cv2.line(img, (-2187, 266), (-2187, 266), (200, 200, 0), 30)
cv2.line(img, (1100, 738), (1100, 738), (200, 200, 0), 30)
cv2.imwrite('./graytest3/test.jpg', img)

Image's shape is 1012, 1920


True